In [1]:
import numpy as np
import pandas as pd

In [2]:
events_df = pd.read_csv("./events.csv")

In [3]:
events_df.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [4]:
print(events_df.columns)

Index(['app_id', 'session_id', 'event', 'event_timestamp', 'event_value',
       'user_id_hash'],
      dtype='object')


In [5]:
events_df['if_purchase'] = events_df['event'].map(lambda x: 1 if x=='8' else 0)

In [6]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [7]:
le_user_id = preprocessing.LabelEncoder()
le_app_id = preprocessing.LabelEncoder()
le_session_id = preprocessing.LabelEncoder()

In [8]:
le_user_id.fit(events_df['user_id_hash'])
le_app_id.fit(events_df['app_id'])
le_session_id.fit(events_df['session_id'])

LabelEncoder()

In [9]:
events_df['user_id_hash'] = le_user_id.transform(events_df['user_id_hash'])
events_df['app_id'] = le_app_id.transform(events_df['app_id'])
events_df['session_id'] = le_session_id.transform(events_df['session_id'])

In [10]:
events_df['user_id_hash'] = events_df['user_id_hash'].astype('uint32')
events_df['app_id'] = events_df['app_id'].astype('uint8')
events_df['session_id'] = events_df['session_id'].astype('uint32')
events_df['if_purchase'] = events_df['if_purchase'].astype('uint8')

In [11]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 7 columns):
app_id             uint8
session_id         uint32
event              object
event_timestamp    int64
event_value        float64
user_id_hash       uint32
if_purchase        uint8
dtypes: float64(1), int64(1), object(1), uint32(2), uint8(2)
memory usage: 3.5+ GB


In [ ]:
events_df['date'] = pd.to_datetime(events_df['event_timestamp'], unit='ms')
events_df.drop(['events_timestamp'], axis=1)

In [13]:
events_df.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash,if_purchase,date
0,0,3018349,45,1542215397132,0.0,372190,0,2018-11-14 17:09:57.132
1,0,3018349,45,1542215484895,0.0,372190,0,2018-11-14 17:11:24.895
2,0,4175808,.m5100869650219008,1541124410372,0.0,372190,0,2018-11-02 02:06:50.372
3,0,1196154,4,1543713091129,0.0,372190,0,2018-12-02 01:11:31.129
4,0,1196154,6,1543713093116,0.0,372190,0,2018-12-02 01:11:33.116


In [19]:
events_df.date.nunique()

109625016

In [14]:
events_df['day'] = events_df['date'].map(lambda dt: dt.date())

In [15]:
events_df.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash,if_purchase,date,day
0,0,3018349,45,1542215397132,0.0,372190,0,2018-11-14 17:09:57.132,2018-11-14
1,0,3018349,45,1542215484895,0.0,372190,0,2018-11-14 17:11:24.895,2018-11-14
2,0,4175808,.m5100869650219008,1541124410372,0.0,372190,0,2018-11-02 02:06:50.372,2018-11-02
3,0,1196154,4,1543713091129,0.0,372190,0,2018-12-02 01:11:31.129,2018-12-02
4,0,1196154,6,1543713093116,0.0,372190,0,2018-12-02 01:11:33.116,2018-12-02


In [16]:
user_day_purchase = events_df.groupby(['user_id_hash', 'day'])['if_purchase'].sum()

In [17]:
user_day_event = events_df.groupby(['user_id_hash', 'day'])['if_purchase'].count()

In [32]:
user_day_purchase[0]

day
2018-11-10    0
2018-11-11    7
2018-11-12    1
2018-11-13    0
2018-11-17    0
2018-12-13    0
Name: if_purchase, dtype: uint8

In [39]:
user_day_event[0]

day
2018-11-10    103
2018-11-11    232
2018-11-12    170
2018-11-13      2
2018-11-17      1
2018-12-13     89
Name: if_purchase, dtype: int64

In [18]:
purchase_ratio_df = pd.DataFrame({'purchase_cnt': user_day_purchase, 'event_cnt': user_day_event}).reset_index()

In [19]:
purchase_ratio_df['purchase_ratio'] = purchase_ratio_df['purchase_cnt'] / purchase_ratio_df['event_cnt']

In [20]:
purchase_ratio_df.head()

,user_id_hash,day,purchase_cnt,event_cnt,purchase_ratio
0,0,2018-11-10,0,103,0.000000
1,0,2018-11-11,7,232,0.030172
2,0,2018-11-12,1,170,0.005882
3,0,2018-11-13,0,2,0.000000
4,0,2018-11-17,0,1,0.000000


In [21]:
pivot_df = purchase_ratio_df.pivot(index='user_id_hash', columns='day', values='purchase_cnt')

In [22]:
pivot_df = pivot_df.fillna(0)

In [23]:
pivot_df

day,2018-10-01,2018-10-02,2018-10-03,2018-10-04,2018-10-05,2018-10-06,2018-10-07,2018-10-08,2018-10-09,2018-10-10,...,2018-12-05,2018-12-06,2018-12-07,2018-12-08,2018-12-09,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14
user_id_hash,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
pivot_df.values.shape

(621001, 75)

In [109]:
import scipy

In [110]:
pivot_sparse_mat = scipy.sparse.csr_matrix(pivot_df.values)

In [111]:
pivot_sparse_mat

<621001x75 sparse matrix of type '<class 'numpy.float64'>'
	with 117320 stored elements in Compressed Sparse Row format>

In [112]:
print(pivot_sparse_mat)

  (0, 41)	7.0
  (0, 42)	1.0
  (1, 52)	1.0
  (124, 66)	3.0
  (124, 67)	1.0
  (124, 68)	1.0
  (124, 71)	1.0
  (124, 74)	1.0
  (170, 57)	2.0
  (170, 69)	1.0
  (172, 13)	1.0
  (172, 15)	1.0
  (238, 28)	2.0
  (238, 29)	1.0
  (280, 2)	1.0
  (280, 14)	2.0
  (280, 15)	1.0
  (280, 16)	1.0
  (280, 18)	1.0
  (288, 44)	2.0
  (288, 45)	3.0
  (288, 46)	1.0
  (288, 47)	2.0
  (288, 48)	3.0
  (288, 49)	2.0
  :	:
  (620805, 36)	1.0
  (620805, 37)	3.0
  (620805, 38)	2.0
  (620805, 39)	9.0
  (620835, 23)	2.0
  (620835, 25)	1.0
  (620846, 46)	1.0
  (620852, 28)	2.0
  (620852, 29)	1.0
  (620852, 30)	2.0
  (620862, 61)	1.0
  (620871, 49)	2.0
  (620900, 12)	4.0
  (620907, 8)	12.0
  (620933, 13)	1.0
  (620933, 18)	1.0
  (620936, 18)	1.0
  (620936, 26)	3.0
  (620936, 27)	1.0
  (620936, 28)	1.0
  (620960, 36)	2.0
  (620960, 37)	4.0
  (620960, 38)	1.0
  (620982, 52)	1.0
  (620995, 6)	1.0


In [113]:
arr = pivot_sparse_mat.toarray()

In [114]:
sparsity = 1.0 - np.count_nonzero(arr) / arr.size

In [115]:
sparsity

0.9974810561228297

In [193]:
sample_submission_df = pd.read_csv("./sample_submission_2.csv")

In [104]:
sample_submission_df.head(10)

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.01,0.02
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.01,0.02
5,7f7e3c5da13a555b2c2047c2c94b45d289ffa8083908cb...,0.01,0.02
6,28b5edd0510e8517462bae9d67a99f272d8ccc8ba2f79d...,0.01,0.02
7,b9433b9c4d559218e64469b6807cb516a5866572c1c30b...,0.01,0.02
8,607da741597a3b15fecf9a30c76e2e418b629c975e069a...,0.01,0.02
9,455ff7334b87646d3ecc05d2e0ef5a8d99dc8fa6573e54...,0.01,0.02


In [25]:
pivot_ratio_df = purchase_ratio_df.pivot(index='user_id_hash', columns='day', values='purchase_ratio')

In [26]:
pivot_ratio_df = pivot_ratio_df.fillna(0)

In [64]:
X_train = pd.concat([np.sum(pivot_df.iloc[:,0:7], axis=1) > 0, 
                     np.sum(pivot_df.iloc[:,7:14], axis=1) > 0, 
                     np.sum(pivot_df.iloc[:,14:21], axis=1) > 0], axis=1)
y_train = np.sum(pivot_ratio_df.iloc[:,21:28], axis=1) > 0

In [129]:
# X_train = pivot_df.iloc[:,0:7]
# y_train = np.sum(pivot_ratio_df.iloc[:,7:14], axis=1)

In [28]:
from sklearn.tree import DecisionTreeClassifier

In [29]:
dtc1 = DecisionTreeClassifier(random_state=0, criterion='entropy', max_depth=3, min_samples_leaf=5)

In [30]:
dtc1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [31]:
y_train_pred = dtc1.predict(X_train)

In [32]:
y_train_pred

array([False, False, False, ..., False, False, False])

In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
roc_auc_score(y_train_pred, y_train)

0.7455333218392045

In [ ]:
# X_test = pivot_df.iloc[:, 7:14]
# y_test = np.sum(pivot_ratio_df.iloc[:,14:28], axis=1)

In [35]:
import xgboost as xgb

In [36]:
xgb_clf = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

In [37]:
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [38]:
y_xgb_train_pred = xgb_clf.predict(X_train)

In [39]:
np.count_nonzero(y_xgb_train_pred)

379

In [40]:
roc_auc_score(y_xgb_train_pred, y_train)

0.7455333218392045

In [41]:
pivot_df.shape

(621001, 75)

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
y_lr_train_pred = lr.predict(X_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [44]:
roc_auc_score(y_lr_train_pred, y_train)

0.7322428804034332

In [45]:
from sklearn.model_selection import StratifiedKFold

In [46]:
def stacking(model, train, y, test, n_fold):
    folds = StratifiedKFold(n_splits=n_fold, random_state=0)
    test_pred = np.empty((test.shape[0], 1), float)
    train_pred = np.empty((0, 1), float)
    
    for train_ind, val_ind in folds.split(train, y):
        x_train, x_val = train.iloc[train_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]
        
        model.fit(x_train, y_train)
        train_pred = np.append(train_pred, model.predict(x_val))
        test_pred = np.append(test_pred, model.predict(test))
        
    return test_pred.reshape(-1, 1), train_pred

In [81]:
train_batch_size = 3

In [90]:
d_trees = []
roc_auc_scores = []
for i in range(0, int(np.floor(pivot_df.shape[1]) / 7) - (train_batch_size)):
    dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5)
    num = i*7
    X_train = pd.concat([np.sum(pivot_df.iloc[:,num:(num+7)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+7):(num+14)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+14):(num+21)], axis=1) > 0], axis=1)
    if i < 7:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):(num+28)], axis=1) > 0
    else:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):], axis=1) > 0
    dtc.fit(X_train, y_train)
    try:
        y_pred = dtc.predict(X_train)
        score = roc_auc_score(y_pred, y_train)
        d_trees.append(dtc)
        roc_auc_scores.append(score)
    except ValueError:
        pass

In [91]:
roc_auc_scores

[0.7455333218392045, 0.7586229466675629, 0.7653695159228449]

In [92]:
num

42

In [93]:
75 - 63

12

In [94]:
X_val = pd.concat([np.sum(pivot_df.iloc[:,63:66], axis=1) > 0, 
                   np.sum(pivot_df.iloc[:,66:69], axis=1) > 0, 
                   np.sum(pivot_df.iloc[:,69:72], axis=1) > 0], axis=1)
y_val = np.sum(pivot_ratio_df.iloc[:,72:75], axis=1) > 0

In [101]:
l_regressions = []
lroc_auc_scores = []
for i in range(0, int(np.floor(pivot_df.shape[1]) / 7) - (train_batch_size)):
    lr = LogisticRegression()
    num = i*7
    X_train = pd.concat([np.sum(pivot_df.iloc[:,num:(num+7)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+7):(num+14)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+14):(num+21)], axis=1) > 0], axis=1)
    if i < 7:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):(num+28)], axis=1) > 0
    else:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):], axis=1) > 0
    lr.fit(X_train, y_train)
    try:
        y_pred = lr.predict(X_train)
        score = roc_auc_score(y_pred, y_train)
        l_regressions.append(lr)
        lroc_auc_scores.append(score)
    except ValueError:
        pass

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [102]:
lroc_auc_scores

[0.7322428804034332,
 0.7361866576133365,
 0.7353752302169487,
 0.7354876071975534,
 0.7406133704637319,
 0.7400107193116467,
 0.7483696994830114]

In [104]:
from sklearn.ensemble import RandomForestClassifier

In [105]:
rf_trees = []
rf_roc_auc_scores = []
for i in range(0, int(np.floor(pivot_df.shape[1]) / 7) - (train_batch_size)):
    rfc = RandomForestClassifier(max_depth=10, n_estimators=3, max_features=1)
    num = i*7
    X_train = pd.concat([np.sum(pivot_df.iloc[:,num:(num+7)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+7):(num+14)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+14):(num+21)], axis=1) > 0], axis=1)
    if i < 7:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):(num+28)], axis=1) > 0
    else:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):], axis=1) > 0
    rfc.fit(X_train, y_train)
    try:
        y_pred = rfc.predict(X_train)
        score = roc_auc_score(y_pred, y_train)
        rf_trees.append(rfc)
        rf_roc_auc_scores.append(score)
    except ValueError:
        pass

In [106]:
rf_roc_auc_scores

[0.7586229466675629, 0.7653695159228449]

In [107]:
from sklearn.ensemble import AdaBoostClassifier

In [219]:
ada_trees = []
ada_roc_auc_scores = []
for i in range(0, int(np.floor(pivot_df.shape[1]) / 7) - (train_batch_size)):
    ada = AdaBoostClassifier()
    num = i*7
    X_train = pd.concat([np.sum(pivot_df.iloc[:,num:(num+7)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+7):(num+14)], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,(num+14):(num+21)], axis=1) > 0], axis=1)
    if i < 7:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):(num+28)], axis=1) > 0
    else:
        y_train = np.sum(pivot_ratio_df.iloc[:,(num+21):], axis=1) > 0
    ada.fit(X_train, y_train)
    try:
        y_pred = ada.predict(X_train)
        score = roc_auc_score(y_pred, y_train)
        ada_trees.append(ada)
        ada_roc_auc_scores.append(score)
    except ValueError:
        pass

In [220]:
ada_roc_auc_scores

[0.6957922972875613,
 0.7088059757296896,
 0.7034801152584387,
 0.7014377278165639,
 0.6949369054186992,
 0.7074016022974297,
 0.7146981997596763]

In [126]:
X_val = pivot_df.iloc[:,70:73] > 0
y_val = np.sum(pivot_ratio_df.iloc[:,73:], axis=1) > 0

In [143]:
kfold = StratifiedKFold(n_splits=10, random_state=0)

In [128]:
for i in range(len(d_trees)):
    print(f"decision_tree_{i}")

decision_tree_0
decision_tree_1
decision_tree_2


In [141]:
estimators = []
for i in range(len(d_trees)):
    estimators.append((f"decision_tree_{i}", d_trees[i]))
for i in range(len(l_regressions)):
    estimators.append((f"logistic_{i}", l_regressions[i]))
for i in range(len(rf_trees)):
    estimators.append((f"random_forest_{i}", rf_trees[i]))
for i in range(len(ada_trees)):
    estimators.append((f"ada_boost_{i}", ada_trees[i]))

In [144]:
from sklearn.ensemble import VotingClassifier

In [161]:
ensemble = VotingClassifier(estimators, voting='soft')

In [162]:
X_train_new = pd.concat([np.sum(pivot_df.iloc[:,42:49], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,49:56], axis=1) > 0, 
                         np.sum(pivot_df.iloc[:,56:63], axis=1) > 0], axis=1)
y_train_new = np.sum(pivot_ratio_df.iloc[:,63:70], axis=1) > 0

In [163]:
ensemble.fit(X_train_new, y_train_new)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

VotingClassifier(estimators=[('decision_tree_0', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [164]:
y_val_pred = ensemble.predict(X_val)
roc_auc_score(y_val_pred, y_val)

0.8494205001886275

In [165]:
y_val_pred = ensemble.predict(X_train_new)
roc_auc_score(y_val_pred, y_train_new)

0.7653695159228449

### User Purchase Binary 7 Days

In [168]:
X_test = pd.concat([np.sum(pivot_df.iloc[:,54:61], axis=1) > 0, 
                    np.sum(pivot_df.iloc[:,61:68], axis=1) > 0, 
                    np.sum(pivot_df.iloc[:,68:75], axis=1) > 0], axis=1)

In [171]:
submission_df = pd.DataFrame({'user_id_hash':le_user_id.inverse_transform(pivot_df.index)})

In [178]:
submission_df['user_purchase_binary_7_days'] = ensemble.predict_proba(X_test)[:,1]

### User Purchase Binary 14 Days

In [188]:
X_test = pd.concat([np.sum(pivot_df.iloc[:,33:47], axis=1) > 0, 
                    np.sum(pivot_df.iloc[:,47:61], axis=1) > 0, 
                    np.sum(pivot_df.iloc[:,61:75], axis=1) > 0], axis=1)

In [189]:
submission_df['user_purchase_binary_14_days'] = ensemble.predict_proba(X_test)[:,1]

In [194]:
sample_submission_df.shape

(312568, 3)

In [203]:
sub_df = submission_df[submission_df['user_id_hash'].isin(sample_submission_df['user_id_hash'])]

In [209]:
sub_df.shape

(312316, 3)

In [207]:
sub_notin_df = sample_submission_df[~sample_submission_df['user_id_hash'].isin(submission_df['user_id_hash'])]

In [208]:
sub_notin_df.shape

(252, 3)

In [211]:
sub_notin_df['user_purchase_binary_7_days'] = np.mean(sub_df['user_purchase_binary_7_days'])
sub_notin_df['user_purchase_binary_14_days'] = np.mean(sub_df['user_purchase_binary_14_days'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [214]:
sub_df_final = pd.concat([sub_df, sub_notin_df], axis=0)

In [217]:
sub_df_final.shape

(312568, 3)

In [216]:
sub_df_final.to_csv('submission.csv', encoding='utf-8', index=False)

### Try with more features, 10 features now

In [295]:
X_train_7d = pd.concat([#np.sum(pivot_df.iloc[:,0:7], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,5:12], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,12:19], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,19:26], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,26:33], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,33:40], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,40:47], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,47:54], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,54:61], axis=1) > 0,
                        np.sum(pivot_df.iloc[:,61:68], axis=1) > 0], axis=1)
y_train = np.sum(pivot_ratio_df.iloc[:,68:], axis=1) > 0

In [296]:
from sklearn.ensemble import GradientBoostingClassifier

## Gradient Boosting

In [305]:
gbc = GradientBoostingClassifier(n_estimators=20)
gbc.fit(X_train_7d, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [272]:
np.count_nonzero(X_train)

21246

In [263]:
np.count_nonzero(y_train_pred)

1242002

In [274]:
y_train_pred

array([0.02994381, 0.04969869, 0.00114128, ..., 0.00114128, 0.00114128,
       0.00114128])

In [307]:
y_train_pred = gbc.predict_proba(X_train_7d) [:,1]

In [308]:
roc_auc_score(y_train, y_train_pred)

0.9204652777879387

## Random Forest

In [313]:
rfc = RandomForestClassifier(max_depth=10, n_estimators=3, max_features=1)
rfc.fit(X_train_7d, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [314]:
y_train_pred_rf = rfc.predict_proba(X_train_7d)[:,1]

In [315]:
roc_auc_score(y_train, y_train_pred_rf)

0.9205137027281287

## Logistic Regression

In [316]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train_7d, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [319]:
y_train_pred_lr = lr.predict_proba(X_train_7d)[:,1]

In [320]:
roc_auc_score(y_train, y_train_pred_lr)

0.9214332270333144

### Making prediction for 7 days

In [327]:
estimators = []

In [328]:
gbc = GradientBoostingClassifier(n_estimators=20)
rfc = RandomForestClassifier(max_depth=10, n_estimators=3, max_features=1)
lr = LogisticRegression()

In [329]:
estimators.append(('random_forest', rfc))
estimators.append(('gradient_boosting', gbc))
estimators.append(('logistic_regression', lr))

In [334]:
vc = VotingClassifier(estimators=estimators, voting='soft')

In [335]:
vc.fit(X_train_7d, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            mi...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [338]:
y_train_pred_vc = vc.predict_proba(X_train_7d)[:,1]

In [339]:
roc_auc_score(y_train, y_train_pred_vc)

0.9217804036078248

In [342]:
sub_in_df = pd.DataFrame({'user_id_hash':le_user_id.inverse_transform(pivot_df.index)})

In [351]:
temp = pd.concat([X_train_7d, y_train], axis=1)
temp.columns = range(temp.shape[1])
X_test = temp.iloc[:,1:]

In [353]:
sub_in_df['user_purchase_binary_7_days'] = vc.predict_proba(X_test)[:,1]

### Making predictions for 14 days

In [357]:
X_train_14d = pd.concat([#np.sum(pivot_df.iloc[:,0:7], axis=1) > 0,
                         np.sum(pivot_df.iloc[:,5:19], axis=1) > 0,
                         np.sum(pivot_df.iloc[:,19:33], axis=1) > 0,
                         np.sum(pivot_df.iloc[:,33:47], axis=1) > 0,
                         np.sum(pivot_df.iloc[:,47:61], axis=1) > 0], axis=1)
y_train_14d = np.sum(pivot_ratio_df.iloc[:,61:], axis=1) > 0

In [358]:
rfc = RandomForestClassifier(max_depth=10, n_estimators=3, max_features=1)
rfc.fit(X_train_14d, y_train_14d)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [360]:
y_train_pred_rf_14d = rfc.predict_proba(X_train_14d)[:,1]
roc_auc_score(y_train_14d, y_train_pred_rf_14d)

0.8577465045390567

In [362]:
estimators = []
gbc = GradientBoostingClassifier(n_estimators=20)
rfc = RandomForestClassifier(max_depth=10, n_estimators=3, max_features=1)
lr = LogisticRegression()
estimators.append(('random_forest', rfc))
estimators.append(('gradient_boosting', gbc))
estimators.append(('logistic_regression', lr))

In [363]:
vc_14d = VotingClassifier(estimators=estimators, voting='soft')

In [364]:
vc_14d.fit(X_train_14d, y_train_14d)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            mi...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [366]:
y_train_pred_vc_14d = vc_14d.predict_proba(X_train_14d)[:,1]
roc_auc_score(y_train_14d, y_train_pred_vc_14d)

0.8577338529625576

In [367]:
temp = pd.concat([X_train_14d, y_train_14d], axis=1)
temp.columns = range(temp.shape[1])
X_test_14d = temp.iloc[:,1:]

In [369]:
sub_in_df['user_purchase_binary_14_days'] = vc_14d.predict_proba(X_test_14d)[:,1]

In [394]:
sub_df = sub_in_df[sub_in_df['user_id_hash'].isin(sample_submission_df['user_id_hash'])]

In [395]:
sub_df.shape

(312316, 3)

In [396]:
sub_notin_df = sample_submission_df[~sample_submission_df['user_id_hash'].isin(sub_in_df['user_id_hash'])]

In [397]:
sub_notin_df.shape

(252, 3)

In [398]:
sub_notin_df['user_purchase_binary_7_days'] = np.mean(sub_df['user_purchase_binary_7_days'])
sub_notin_df['user_purchase_binary_14_days'] = np.mean(sub_df['user_purchase_binary_14_days'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [399]:
sub_df_final = pd.concat([sub_df, sub_notin_df], axis=0)
sub_df_final.shape

(312568, 3)

In [402]:
sub_df_final.to_csv('submission.csv', encoding='utf-8', index=False)